# Working with Data

The intent of this tutorial is to help familiarize yourself with browsing for data that will be used along with an application to generate data by submitting a job. Job submission will be covered in the next tutorial. Run each cell in order (shift-enter). The notes will indicate when you need to edit code to customize things (e.g., to indicate a data collection)vs. being prompted by running the cell (e.g. for your username and password).

In [ ]:
import requests
import getpass
import json
from IPython.display import JSON

First we need some pre-defined environment variables

In [ ]:
# This portion of the code is env specific for Dev, Test, Ops, etc. 
# define the environment as our test venue
env = {
    "clientId":"71894molftjtie4dvvkbjeard0",
    "url":"https://58nbcawrvb.execute-api.us-west-2.amazonaws.com/test/"
      }

# The auth_json is template for authorizing with AWS Cognito for a token that can be used for calls to the data service.
# For now this is just an empty data structure. You will be prompted for your username and password in a few steps.
auth_json = '''{
     "AuthParameters" : {
        "USERNAME" : "",
        "PASSWORD" : ""
     },
     "AuthFlow" : "USER_PASSWORD_AUTH",
     "ClientId" : ""
  }'''

### Authentication Code

The below method is a helper function for getting an access token for accessing Unity SDS services. You must pass the token along with any API requests in order to access the various Unity SDS services.

In [ ]:
# This method is used for taking a username and password and client ID and fetching a cognito token
def get_token(username, password, clientID):
    aj = json.loads(auth_json)
    aj['AuthParameters']['USERNAME'] = username
    aj['AuthParameters']['PASSWORD'] = password
    aj['ClientId'] =clientID 
    token = None
    try:
        response = requests.post('https://cognito-idp.us-west-2.amazonaws.com', headers={"Content-Type":"application/x-amz-json-1.1", "X-Amz-Target":"AWSCognitoIdentityProviderService.InitiateAuth"}, json=aj)
        token = response.json()['AuthenticationResult']['AccessToken']
    except:
        print("Error, check username and password and try again.")
    return token

### Prompt for your Unity username and password

These are required to get the token (described above) to connect to the data services.

In [ ]:
print("Please enter your username...")
user_name = input()

print("Please enter your password...")
password = getpass.getpass()

In [ ]:
token = get_token(user_name, password, env['clientId'])

if(token):
    print("Token received.")

## List Available Data Collections in the Unity System

Data is organized into Collections. Any particular data file will be in at least one Collection.

In [ ]:
# The DAPA-request endpoint to retrieve collections is the base URL plus the following:
url = env['url'] + "am-uds-dapa/collections"

# Make a GET request at the URL you have constructed, using your access token
response = requests.get(url, headers={"Authorization": "Bearer " + token})

print ("Data Collections at " + url)
# To see raw JSON of the API response, uncomment this line:
#print(json.dumps(response.json()))

features = response.json()['features']

for data_set in features:
   print(data_set['id'])

print("\nFull JSON response object:")
JSON(response.json())

## Given a collection (above), List the files within that collection

Executing this cell will retrieve all the files in a Collection defined by the data_set variable. Then it will print out the name and href location of each (up to a limit defined in this code block).

To see a different data Collection, change the data_set variable to one of the other Collections you found in the step above. If you would like to limit your query to something other than 100 files, change the value in the params.append() call.

In [ ]:
data_set = "SNDR_SNPP_ATMS_L1B_OUTPUT___1"
url = env['url'] + "am-uds-dapa/collections/"+data_set+"/items"

params = []
params.append(("limit", 20))

response = requests.get(url, headers={"Authorization": "Bearer " + token}, params=params)

print(f"Total number of files: {response.json()['numberMatched']}")
print("File IDs, titles, and hrefs in Collection " + data_set + "\n")

features = response.json()['features']

for data_file in features: {
   print(data_file['id']),
   print(data_file['assets']['metadata__data']['title']),
   print(data_file['assets']['metadata__data']['href']),
   print("")
}

print("Full JSON response object:")
JSON(response.json())


## Filter the results above by time

The standards-based API used by the Unity SDS Data Store, DAPA, has a variety of filtering options. Currently we have implemented a time-based filter. See more about the Data Access and Processing API at: https://docs.ogc.org/per/20-025r1.html#_dapa_overview

This cell will filter the full list of files in the Collection with ID = data_set by a start and end time defined by the datetime parameter.

In [ ]:
data_set = "SNDR_SNPP_ATMS_L1B_OUTPUT___1"
url = env['url'] + "am-uds-dapa/collections/"+data_set+"/items"
# the datetime,limit, and offset are included due to a current bug in the API Gatway setting these values to 'none'.
# Example date/time params

params = []
#add a datetime to your request
params.append(("datetime", "2000-11-01T00:00:00Z/2022-11-01T02:31:12Z"))

# limit - how many results to return in a single request
params.append(("limit", 10))

response = requests.get(url, headers={"Authorization": "Bearer " + token}, params=params)

print(f"Total number of files: {response.json()['numberMatched']}")
print("File IDs, datetimes, and hrefs in Collection " + data_set + "\n")

features = response.json()['features']
while len(features) > 0:
    for data_file in features: {
       print(data_file['id']),
       print(data_file['properties']['created']),
       print(data_file['assets']['metadata__data']['href']),
       print("")
    }
    # Get the next page of results
    response = requests.get(next(item for item in response.json()['links'] if item['rel'] == 'next')['href'], headers={"Authorization": "Bearer " + token}, params=params)
    features = response.json()['features']

## Explore on your own

Given the endpoints above for finding collections and then finding data within those collections, try to craft a query by copying cells to find data from one of the other collections in the list above.

Some things to try:

* Find data in the Unity system for the L0_SNPP_EphAtt product type
* Find data in the Unity system for the L1 SounderSIPS 
* Filter the collections above on a numer



## Credential-less data download

When accessing data stores within the same venue, you'll be able to download data from S3 without credentials. 

**Note**, the following libraries are needed for this, and the below command can be run in a jupyter-terminal to install them:

```
conda install xarray netcdf4 hdf5 boto3 matplotlib
```


In [1]:
import boto3

In [2]:
s3 = boto3.client('s3')
s3.download_file('uds-test-cumulus-protected', 'SNDR_SNPP_ATMS_L1A___1/SNDR.SNPP.ATMS.L1A.nominal2.04.nc', 'test_file11.nc')

In [3]:
import xarray as xr
ds = xr.open_dataset('test_file11.nc')
ds

<xarray.Dataset>
Dimensions:                    (atrack: 135, attitude: 3, band: 5, bbxtrack: 4, channel: 22, engtrack: 45, fov_poly: 8, kav4wtemp: 8, shelftemp: 4, spacextrack: 4, spatial: 3, utc_tuple: 8, wg4wtemp: 7, xtrack: 96)
Coordinates:
    lat                        (atrack, xtrack) float32 ...
    lon                        (atrack, xtrack) float32 ...
    subsat_lat                 (atrack) float32 ...
    subsat_lon                 (atrack) float32 ...
    attitude_lbl               (attitude) object ...
    spatial_lbl                (spatial) object ...
    utc_tuple_lbl              (utc_tuple) object ...
    band_lat                   (atrack, xtrack, band) float32 ...
    band_lon                   (atrack, xtrack, band) float32 ...
    band_lbl                   (band) object ...
    kkav4wtemp_lbl             (kav4wtemp) object ...
    wg4wtemp_lbl               (wg4wtemp) object ...
  * channel                    (channel) float32 1.0 2.0 3.0 ... 20.0 21.0 22.0
    shelftemp_lbl              (shelftemp) object ...
Dimensions without coordinates: atrack, attitude, band, bbxtrack, engtrack, fov_poly, kav4wtemp, shelftemp, spacextrack, spatial, utc_tuple, wg4wtemp, xtrack
Data variables:
    obs_id                     (atrack, xtrack) object ...
    instrument_state           (atrack, xtrack) float32 ...
    obs_time_tai93             (atrack, xtrack) datetime64[ns] ...
    obs_time_utc               (atrack, xtrack, utc_tuple) float32 ...
    lat_geoid                  (atrack, xtrack) float32 ...
    lon_geoid                  (atrack, xtrack) float32 ...
    lat_bnds                   (atrack, xtrack, fov_poly) float32 ...
    lon_bnds                   (atrack, xtrack, fov_poly) float32 ...
    land_frac                  (atrack, xtrack) float32 ...
    surf_alt                   (atrack, xtrack) float32 ...
    surf_alt_sdev              (atrack, xtrack) float32 ...
    sun_glint_lat              (atrack) float32 ...
    sun_glint_lon              (atrack) float32 ...
    sol_zen                    (atrack, xtrack) float32 ...
    sol_azi                    (atrack, xtrack) float32 ...
    sun_glint_dist             (atrack, xtrack) float32 ...
    view_ang                   (atrack, xtrack) float32 ...
    sat_zen                    (atrack, xtrack) float32 ...
    sat_azi                    (atrack, xtrack) float32 ...
    sat_range                  (atrack, xtrack) float32 ...
    asc_flag                   (atrack) float32 ...
    scan_mid_time              (atrack) datetime64[ns] ...
    sat_alt                    (atrack) float32 ...
    sat_pos                    (atrack, spatial) float32 ...
    sat_vel                    (atrack, spatial) float32 ...
    sat_att                    (atrack, attitude) float32 ...
    moon_ang                   (atrack, spacextrack) float32 ...
    local_solar_time           (atrack, xtrack) timedelta64[ns] ...
    mean_anom_wrt_equat        (atrack) float32 ...
    sat_sol_zen                (atrack) float32 ...
    sat_sol_azi                (atrack) float32 ...
    asc_node_lon               float32 ...
    asc_node_tai93             datetime64[ns] ...
    asc_node_local_solar_time  timedelta64[ns] ...
    solar_beta_angle           float32 ...
    geo_qualflag               (atrack, xtrack) float64 ...
    band_lat_bnds              (atrack, xtrack, band, fov_poly) float32 ...
    band_lon_bnds              (atrack, xtrack, band, fov_poly) float32 ...
    band_land_frac             (atrack, xtrack, band) float32 ...
    band_surf_alt              (atrack, xtrack, band) float32 ...
    band_geoloc_chan           (band) float32 ...
    sci_err_status             (atrack, xtrack) float32 ...
    sci_status_code            (atrack, xtrack) float32 ...
    sci_scan_time              (atrack, xtrack) datetime64[ns] ...
    sci_scan_ang               (atrack, xtrack) float32 ...
    sci_rad_cnts               (atrack, xtrack, channel) float32 ...
    hot_obs_id        

In [ ]:
ds.band_surf_alt.plot()